In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import joblib
import time
import sys
import os
import psutil

# Set random seed for reproducibility
np.random.seed(42)

# Print system and XGBoost info
print(f"XGBoost version: {xgb.__version__}")
print(f"CPU cores: {psutil.cpu_count(logical=True)}, Physical cores: {psutil.cpu_count(logical=False)}")
print(f"Available RAM: {psutil.virtual_memory().available / 1024 / 1024:.2f} MB")

# Step 1: Load the full dataset
try:
    df = pd.read_csv('dart_mwendokasi_data.csv')
    print(f"Using full dataset with {df.shape[0]} records.")
    print(f"Dataset size: {df.shape}")
except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure 'dart_mwendokasi_data.csv' is in the working directory.")
    sys.exit(1)
except Exception as e:
    print(f"Error loading dataset: {e}")
    sys.exit(1)

# Step 2: Preprocess the data
try:
    df = df.dropna()
    print("Data cleaned by removing NaN values.")
    
    # Convert original date to ordinal numbers
    df['date_ordinal'] = pd.to_datetime(df['date'], errors='coerce').map(lambda x: x.toordinal() if pd.notna(x) else 0)
    print("Original date converted to ordinal numbers.")

    # Convert date to day name for encoding
    df['day'] = pd.to_datetime(df['date'], errors='coerce').dt.day_name()
    print("Date converted to day names for 'day' column.")
    
    # Drop the original date column
    df = df.drop(columns=['date'])
    
    # Encode categorical variables
    encoders = {}
    for column in ['day', 'weather', 'peak_hours', 'weekends', 'holidays']:
        encoders[column] = LabelEncoder()
        df[column] = encoders[column].fit_transform(df[column])
        print(f"{column} encoded. Classes: {encoders[column].classes_}")

    # Preprocess time_value
    def convert_to_minutes(time_str):
        try:
            if pd.isna(time_str):
                return 0
            time_str = str(time_str).strip()
            if not time_str or time_str == ":00":
                return 0
            time_parts = time_str.split(':')
            if len(time_parts) >= 2:
                hour = int(time_parts[0]) if time_parts[0].isdigit() else 0
                minute = int(time_parts[1]) if time_parts[1].isdigit() else 0
                return max(0, min(1440, hour * 60 + minute))
            return 0
        except Exception as e:
            print(f"Error converting time_value '{time_str}': {e}")
            return 0

    df['time_value'] = df['time_value'].apply(convert_to_minutes).astype('int32')  # Use int32 for memory
    print("Time_value converted to minutes since midnight.")
    print(f"Sample time_value values: {df['time_value'].head().tolist()}")

except Exception as e:
    print(f"Error in preprocessing: {e}")
    sys.exit(1)

# Step 3: Split features and target
try:
    X = df.drop(columns=['passengers']).astype('float32')  # Use float32 for memory
    training_feature_order = X.columns.tolist()
    print("Training feature order:", training_feature_order)
    y = df['passengers'].astype('float32')
    print("Features and target split successfully.")
except Exception as e:
    print(f"Error splitting features and target: {e}")
    sys.exit(1)

# Step 4: Train-test split
try:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f'Training set size: {X_train.shape}, Testing set size: {X_test.shape}')
except Exception as e:
    print(f"Error in train-test split: {e}")
    sys.exit(1)

# Step 5: Train XGBoost model with optimized threads
try:
    start_time = time.time()
    n_jobs = 6  # Optimized for 500,000 rows, 8 logical threads
    
    # Convert data types for better performance
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')
    
    # Create two models: one with early stopping for main training, one without for cross-validation
    model_with_early_stopping = xgb.XGBRegressor(
        n_estimators=200,  # Reduced from 400
        max_depth=6,       # Reduced from 8
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        n_jobs=n_jobs,
        tree_method='hist',
        random_state=42,
        enable_categorical=False,
        early_stopping_rounds=10
    )
    
    # Model without early stopping for cross-validation
    model = xgb.XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        n_jobs=n_jobs,
        tree_method='hist',
        random_state=42,
        enable_categorical=False
    )
    
    print(f"Models initialized with n_jobs={n_jobs} threads.")
    
    # Train the model with early stopping
    try:
        model_with_early_stopping.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
            verbose=False
        )
        # Use the early stopped model as the main model
        model = model_with_early_stopping
        print("Model trained with early stopping using XGBoost 3.x API.")
    except Exception as early_stop_error:
        print(f"Early stopping failed: {early_stop_error}")
        print("Training without early stopping...")
        model.fit(X_train, y_train)
        print("Model trained without early stopping.")

    print(f'Training time: {time.time() - start_time:.2f} seconds')
    print(f"Memory usage: {psutil.virtual_memory().used / 1024 / 1024:.2f} MB")
except Exception as e:
    print(f"Error training model: {e}")
    sys.exit(1)

# Step 6: Evaluate the model
try:
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Mean Squared Error (MSE): {mse:.2f}')
    print(f'R² Score: {r2:.4f}')

    # For cross-validation, use XGBoost's native cv function instead of sklearn's cross_val_score
    # This avoids the early stopping issue entirely
    try:
        print("Running cross-validation using XGBoost native CV...")
        dtrain = xgb.DMatrix(X, label=y)
        cv_params = {
            'max_depth': 6,
            'learning_rate': 0.05,
            'objective': 'reg:squarederror',
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.1,
            'reg_lambda': 1.0,
            'seed': 42,
            'nthread': n_jobs
        }
        
        cv_results = xgb.cv(
            cv_params, 
            dtrain, 
            num_boost_round=200, 
            nfold=5, 
            seed=42,
            verbose_eval=False,
            show_stdv=False
        )
        
        # Get the last row of CV results (final scores)
        final_train_rmse = cv_results['train-rmse-mean'].iloc[-1]
        final_test_rmse = cv_results['test-rmse-mean'].iloc[-1]
        
        # Convert RMSE to R² approximation (not exact but gives an idea)
        # R² ≈ 1 - (RMSE²/Var(y))
        y_var = np.var(y)
        cv_r2_approx = 1 - (final_test_rmse**2 / y_var)
        
        print(f'5-Fold CV RMSE (train): {final_train_rmse:.4f}')
        print(f'5-Fold CV RMSE (test): {final_test_rmse:.4f}')
        print(f'5-Fold CV R² (approximation): {cv_r2_approx:.4f}')
        
    except Exception as cv_error:
        print(f"XGBoost native CV failed: {cv_error}")
        print("Skipping cross-validation...")
        
except Exception as e:
    print(f"Error evaluating model: {e}")
    sys.exit(1)

# Step 7: Feature importance
try:
    feature_importance = model.feature_importances_
    for feature, importance in zip(X.columns, feature_importance):
        print(f'Feature: {feature}, Importance: {importance:.4f}')
except Exception as e:
    print(f"Error calculating feature importance: {e}")
    sys.exit(1)

# Step 8: Save the model and encoders
try:
    joblib.dump(model, 'xgboost_model.pkl')
    # Save in native XGBoost format
    model.save_model('xgboost_model.json')  # Using JSON format instead of UBJ for better compatibility
    for column, enc in encoders.items():
        joblib.dump(enc, f'{column}_encoder.pkl')
    print("Model and encoders saved as 'xgboost_model.pkl', 'xgboost_model.json', and individual encoder files")
except Exception as e:
    print(f"Error saving model or encoders: {e}")
    sys.exit(1)

# Optional: Classification for F-score
try:
    y_class = pd.cut(y, bins=[0, 50, 100, float('inf')], labels=['Low', 'Medium', 'High'])
    label_encoder_class = LabelEncoder()
    y_class_encoded = label_encoder_class.fit_transform(y_class)
    X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_class_encoded, test_size=0.2, random_state=42)
    
    model_class = xgb.XGBClassifier(
        n_estimators=100,  # Reduced from 200
        max_depth=6,
        learning_rate=0.1,
        n_jobs=n_jobs,
        tree_method='hist',
        random_state=42
    )
    model_class.fit(X_train_c, y_train_c)
    y_pred_c = model_class.predict(X_test_c)
    
    from sklearn.metrics import classification_report
    print('\nClassification Report (F-score):')
    print(classification_report(y_test_c, y_pred_c, target_names=['Low', 'Medium', 'High']))
    
    # Save classification model
    joblib.dump(model_class, 'xgboost_classifier.pkl')
    joblib.dump(label_encoder_class, 'passenger_class_encoder.pkl')
    
except Exception as e:
    print(f"Error in classification: {e}")

# Function to predict passenger flow
def predict_passenger_flow(date_ordinal, time_minutes, day, weather, peak_hours, weekends, holidays):
    try:
        input_data = pd.DataFrame({
            'date_ordinal': [date_ordinal],
            'time_value': [time_minutes],
            'day': [encoders['day'].transform([day.title()])[0]],
            'weather': [encoders['weather'].transform([weather.title()])[0]],
            'peak_hours': [encoders['peak_hours'].transform([peak_hours.title()])[0]],
            'weekends': [encoders['weekends'].transform([weekends.title()])[0]],
            'holidays': [encoders['holidays'].transform([holidays.title()])[0]]
        }, columns=training_feature_order).astype('float32')
        prediction = model.predict(input_data)
        return prediction[0]
    except Exception as e:
        print(f"Error in prediction: {e}")
        return None

# Test function to evaluate model efficiency
def test_model_efficiency():
    test_cases = [
        (pd.to_datetime('2025-07-23').toordinal(), 8 * 60, 'Wednesday', 'Sunny', 'No', 'No', 'No'),
        (pd.to_datetime('2025-07-23').toordinal(), 17 * 60, 'Friday', 'Rainy', 'Yes', 'No', 'No'),
        (pd.to_datetime('2025-07-26').toordinal(), 12 * 60, 'Sunday', 'Sunny', 'No', 'Yes', 'No'),
    ]
    start_time = time.time()
    for i, (date_ordinal, time_minutes, day, weather, peak_hours, weekends, holidays) in enumerate(test_cases):
        pred = predict_passenger_flow(date_ordinal, time_minutes, day, weather, peak_hours, weekends, holidays)
        print(f"Test case {i+1}: {day} {time_minutes//60}:00, {weather}, Peak: {peak_hours}, Weekend: {weekends}, Holiday: {holidays} -> Predicted passengers: {pred}")
    end_time = time.time()
    print(f"Total time for {len(test_cases)} predictions: {end_time - start_time:.2f} seconds")
    print(f"Average time per prediction: {(end_time - start_time)/len(test_cases):.4f} seconds")

if __name__ == "__main__":
    try:
        today_ordinal = pd.to_datetime('2025-07-23').toordinal()
        print(f"Predicted flow for today at 6 PM: {predict_passenger_flow(today_ordinal, 18 * 60, 'Wednesday', 'Sunny', 'No', 'No', 'No')}")
        print("\nRunning efficiency test...")
        test_model_efficiency()
    except Exception as e:
        print(f"Error in example prediction or test: {e}")

XGBoost version: 3.0.1
CPU cores: 8, Physical cores: 4
Available RAM: 6566.35 MB
Using full dataset with 500000 records.
Dataset size: (500000, 8)
Data cleaned by removing NaN values.
Original date converted to ordinal numbers.
Date converted to day names for 'day' column.
day encoded. Classes: ['Friday' 'Monday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday' 'Wednesday']
weather encoded. Classes: ['Rainy' 'Sunny']
peak_hours encoded. Classes: ['No' 'Yes']
weekends encoded. Classes: ['No' 'Yes']
holidays encoded. Classes: ['No' 'Yes']
Time_value converted to minutes since midnight.
Sample time_value values: [240, 240, 240, 240, 240]
Training feature order: ['day', 'weather', 'time_value', 'peak_hours', 'weekends', 'holidays', 'date_ordinal']
Features and target split successfully.
Training set size: (400000, 7), Testing set size: (100000, 7)
Models initialized with n_jobs=6 threads.
Model trained with early stopping using XGBoost 3.x API.
Training time: 2.52 seconds
Memory usage: 9644.31 MB
